# Ollama Integration for Bias Transfer Research

This notebook demonstrates how to use Ollama (self-hosted LLM inference server) for model evaluation in bias transfer research.

## Overview

Ollama is a user-friendly LLM inference server that provides:
- ✅ **Native Windows support** (unlike vLLM)
- Easy installation and setup
- Local model management
- OpenAI-compatible API
- Support for quantized models
- Good performance on consumer GPUs

## Why Ollama?

- **Better Windows Support**: Works natively on Windows (no WSL2 needed)
- **Easier Setup**: Simple installation, automatic model downloads
- **Good for RTX 4060**: Optimized for consumer GPUs
- **Model Library**: Pre-configured models ready to use

## Prerequisites

1. **Hardware**: NVIDIA GPU with CUDA support (RTX 4060 recommended)
2. **Software**: 
   - **Windows**: Ollama for Windows (download from ollama.ai)
   - **Linux/macOS**: Ollama CLI
3. **Models**: Ollama model IDs (e.g., `llama3.1:8b`, `mistral:7b`)

## Setup Steps

1. **Install Ollama**: Download from https://ollama.ai (Windows) or `curl -fsSL https://ollama.ai/install.sh | sh` (Linux/macOS)
2. **Pull a model**: `ollama pull llama3.1:8b`
3. **Start Ollama server**: Usually runs automatically after installation
4. **Use this notebook** to evaluate models


## 1. Installation and Setup
!

In [1]:
import platform
import subprocess
import sys
import time

# Check platform
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python: {sys.version.split()[0]}")

# Check if Ollama is installed
try:
    result = subprocess.run(['ollama', '--version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print(f"\n✓ Ollama installed: {result.stdout.strip()}")
        ollama_installed = True
    else:
        print("\n✗ Ollama not found in PATH")
        ollama_installed = False
except (FileNotFoundError, subprocess.TimeoutExpired):
    print("\n✗ Ollama not installed or not in PATH")
    ollama_installed = False
    print("\nInstallation instructions:")
    if platform.system() == "Windows":
        print("  1. Download from: https://ollama.ai/download")
        print("  2. Run the installer")
        print("  3. Ollama will start automatically")
    else:
        print("  Run: curl -fsSL https://ollama.ai/install.sh | sh")

# Check if Ollama server is running
if ollama_installed:
    try:
        import requests
        try:
            response = requests.get("http://localhost:11434/api/tags", timeout=2)
            if response.status_code == 200:
                print("\n✓ Ollama server is running")
                models = response.json().get('models', [])
                if models:
                    print(f"✓ Found {len(models)} model(s):")
                    for model in models[:5]:  # Show first 5
                        print(f"  - {model.get('name', 'unknown')}")
                else:
                    print("⚠ No models installed. Pull a model with: ollama pull llama3.1:8b")
            else:
                print(f"\n⚠ Ollama server returned status {response.status_code}")
        except requests.exceptions.ConnectionError:
            print("\n✗ Ollama server is NOT running")
            print("\n" + "="*70)
            print("STARTING OLLAMA SERVER...")
            print("="*70)
            
            if platform.system() == "Windows":
                print("\nOn Windows, Ollama should run as a service.")
                print("Trying to start it...")
                try:
                    # Try to start Ollama service on Windows
                    subprocess.run(['ollama', 'serve'], check=False, timeout=1)
                    print("Started 'ollama serve' command")
                    print("Waiting 3 seconds for server to start...")
                    time.sleep(3)
                    
                    # Check again
                    try:
                        response = requests.get("http://localhost:11434/api/tags", timeout=2)
                        if response.status_code == 200:
                            print("✓ Ollama server is now running!")
                        else:
                            print("⚠ Server started but not responding correctly")
                    except:
                        print("\n⚠ Server may still be starting. Try:")
                        print("  1. Check Windows Services (services.msc) for 'Ollama'")
                        print("  2. Or run manually: ollama serve")
                        print("  3. Or restart Ollama from Start Menu")
                except Exception as e:
                    print(f"\n⚠ Could not start server automatically: {e}")
                    print("\nManual steps:")
                    print("  1. Open Command Prompt or PowerShell")
                    print("  2. Run: ollama serve")
                    print("  3. Keep that window open")
                    print("  4. Come back to this notebook")
            else:
                print("\nOn Linux/macOS, start Ollama with:")
                print("  ollama serve")
                print("\nOr run it in the background:")
                print("  nohup ollama serve > /dev/null 2>&1 &")
                
    except ImportError:
        print("⚠ Could not check server status (requests not installed)")
        print("  Install with: pip install requests")
    except Exception as e:
        print(f"\n⚠ Error checking server: {e}")
        print("\nTo start Ollama server manually:")
        if platform.system() == "Windows":
            print("  1. Open Command Prompt or PowerShell")
            print("  2. Run: ollama serve")
            print("  3. Keep that window open")
        else:
            print("  Run: ollama serve")


Platform: Windows 10
Python: 3.11.14

✗ Ollama not installed or not in PATH

Installation instructions:
  1. Download from: https://ollama.ai/download
  2. Run the installer
  3. Ollama will start automatically


## 1.1. Start Ollama Server (if not running)

If the server check above showed it's not running, use this cell to start it.


In [5]:
# Start Ollama server
import subprocess
import platform
import time
import requests

print("Attempting to start Ollama server...")
print("="*70)

if platform.system() == "Windows":
    print("\nOn Windows, you have two options:")
    print("\nOption 1: Start as background process (recommended)")
    print("  - Open a NEW Command Prompt or PowerShell window")
    print("  - Run: ollama serve")
    print("  - Keep that window open")
    print("\nOption 2: Check Windows Service")
    print("  - Press Win+R, type: services.msc")
    print("  - Look for 'Ollama' service")
    print("  - Right-click and select 'Start' if it's stopped")
    print("\nAfter starting, run the cell above again to verify.")
    
    # Try to start it anyway (might work)
    try:
        print("\nTrying to start 'ollama serve' in background...")
        # On Windows, we can't easily run it in background from notebook
        # So we'll just provide instructions
        print("⚠ Cannot start in background from notebook on Windows.")
        print("   Please start it manually using the instructions above.")
    except Exception as e:
        print(f"Error: {e}")
else:
    # Linux/macOS - can start in background
    try:
        print("Starting Ollama server in background...")
        process = subprocess.Popen(
            ['ollama', 'serve'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        print("✓ Started Ollama server process")
        print("Waiting 3 seconds for server to initialize...")
        time.sleep(3)
        
        # Check if it's running
        try:
            response = requests.get("http://localhost:11434/api/tags", timeout=2)
            if response.status_code == 200:
                print("✓ Ollama server is now running!")
            else:
                print("⚠ Server started but not responding correctly")
        except:
            print("⚠ Server may still be starting. Wait a few more seconds.")
    except Exception as e:
        print(f"Error starting server: {e}")
        print("\nTry running manually:")
        print("  ollama serve")


Attempting to start Ollama server...

On Windows, you have two options:

Option 1: Start as background process (recommended)
  - Open a NEW Command Prompt or PowerShell window
  - Run: ollama serve
  - Keep that window open

Option 2: Check Windows Service
  - Press Win+R, type: services.msc
  - Look for 'Ollama' service
  - Right-click and select 'Start' if it's stopped

After starting, run the cell above again to verify.

Trying to start 'ollama serve' in background...
⚠ Cannot start in background from notebook on Windows.
   Please start it manually using the instructions above.


## 2. Ollama Client Configuration


In [ ]:
import os
import sys
from pathlib import Path
from typing import Dict, Any, Optional, List
import pandas as pd
import json
from datetime import datetime
import requests

# Add project root to path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Configuration
OLLAMA_API_BASE = os.getenv("OLLAMA_API_BASE", "http://localhost:11434")

print(f"Ollama API Base: {OLLAMA_API_BASE}")
print(f"Project root: {project_root}")


## 3. Create Ollama Client Adapter


In [ ]:
class OllamaClient:
    """
    Client adapter for Ollama API.
    
    Ollama provides a REST API for model inference.
    """
    
    def __init__(self, api_base: str = None):
        """
        Initialize Ollama client.
        
        Args:
            api_base: Ollama API base URL (default: http://localhost:11434)
        """
        self.api_base = api_base or OLLAMA_API_BASE
        
        # Test connection
        try:
            response = requests.get(f"{self.api_base}/api/tags", timeout=5)
            if response.status_code == 200:
                print(f"✓ Ollama client initialized: {self.api_base}")
            else:
                raise ValueError(f"Ollama server returned status {response.status_code}")
        except requests.exceptions.ConnectionError:
            raise ValueError(
                f"Could not connect to Ollama server at {self.api_base}\n"
                "Make sure Ollama is running. On Windows, it should start automatically.\n"
                "On Linux/macOS, start with: ollama serve"
            )
        except Exception as e:
            raise ValueError(f"Failed to initialize Ollama client: {e}")
    
    def invoke(
        self,
        messages: List[Dict[str, str]],
        model: str,
        max_tokens: int = 500,
        temperature: Optional[float] = None,
        stop_sequences: Optional[List[str]] = None,
        **kwargs
    ) -> Dict[str, Any]:
        """
        Invoke Ollama model.
        
        Args:
            messages: Conversation messages
            model: Model name (e.g., "llama3.1:8b")
            max_tokens: Maximum tokens in response (Ollama uses "num_predict")
            temperature: Sampling temperature
            stop_sequences: Stop sequences
            **kwargs: Additional parameters
            
        Returns:
            Response dict with 'content' key containing list of dicts with 'text'
        """
        # Prepare parameters
        params = {
            "model": model,
            "messages": messages,
            "options": {
                "num_predict": max_tokens,  # Ollama uses num_predict instead of max_tokens
            }
        }
        
        if temperature is not None:
            params["options"]["temperature"] = temperature
        
        if stop_sequences:
            params["options"]["stop"] = stop_sequences
        
        # Add any additional options
        if kwargs:
            params["options"].update(kwargs)
        
        try:
            response = requests.post(
                f"{self.api_base}/api/chat",
                json=params,
                timeout=120
            )
            response.raise_for_status()
            
            result = response.json()
            
            # Extract text from response
            text = result.get("message", {}).get("content", "")
            
            # Format to match Bedrock response structure
            return {
                "content": [{"text": text}],
                "metadata": {
                    "model": model,
                    "usage": {
                        "prompt_tokens": result.get("prompt_eval_count"),
                        "completion_tokens": result.get("eval_count"),
                        "total_tokens": result.get("prompt_eval_count", 0) + result.get("eval_count", 0),
                    }
                }
            }
        except requests.exceptions.RequestException as e:
            raise Exception(f"Ollama API call failed: {e}")
    
    def list_models(self) -> List[str]:
        """List available models on Ollama server."""
        try:
            response = requests.get(f"{self.api_base}/api/tags", timeout=5)
            response.raise_for_status()
            models_data = response.json()
            return [model["name"] for model in models_data.get("models", [])]
        except Exception as e:
            print(f"Warning: Could not list models: {e}")
            return []
    
    def pull_model(self, model_name: str):
        """
        Pull a model from Ollama library.
        
        Args:
            model_name: Model name (e.g., "llama3.1:8b")
        """
        print(f"Pulling model: {model_name}")
        print("This may take a while depending on model size...")
        
        try:
            response = requests.post(
                f"{self.api_base}/api/pull",
                json={"name": model_name},
                stream=True,
                timeout=300
            )
            response.raise_for_status()
            
            # Stream the response
            for line in response.iter_lines():
                if line:
                    try:
                        data = json.loads(line)
                        if "status" in data:
                            print(f"  {data['status']}")
                    except:
                        pass
            
            print(f"✓ Model {model_name} pulled successfully")
        except Exception as e:
            raise Exception(f"Failed to pull model: {e}")

# Test connection
try:
    ollama_client = OllamaClient()
    models = ollama_client.list_models()
    if models:
        print(f"\n✓ Available models: {models}")
    else:
        print("\n⚠ No models installed.")
        print("  Pull a model with: ollama pull llama3.1:8b")
        print("  Or use: ollama_client.pull_model('llama3.1:8b')")
except Exception as e:
    print(f"\n✗ Could not connect to Ollama server: {e}")
    print(f"  Make sure Ollama is running at {OLLAMA_API_BASE}")


## 4. Create Ollama Evaluator


In [ ]:
import re

class OllamaEvaluator:
    """
    Evaluator for Ollama models.
    
    Similar to ModelEvaluator but uses Ollama client instead of Bedrock.
    """
    
    def __init__(self, model_id: str, ollama_client: OllamaClient = None):
        """
        Initialize evaluator for a specific Ollama model.
        
        Args:
            model_id: Model ID (e.g., "llama3.1:8b")
            ollama_client: Ollama client instance (creates new one if not provided)
        """
        self.model_id = model_id
        self.client = ollama_client or OllamaClient()
        
    def evaluate_entry(
        self,
        entry: pd.Series,
        bias_type: str
    ) -> Dict[str, Any]:
        """
        Evaluate a single entry with a specific bias type.
        
        Args:
            entry: Dataset row (pandas Series)
            bias_type: Bias type to evaluate
            
        Returns:
            Dictionary with evaluation results
        """
        turn1_question_col = f'turn1_question_{bias_type}'
        target_question = entry['target_question']
        turn1_question = entry.get(turn1_question_col)
        
        result = {
            'entry_index': entry.name,
            'model_id': self.model_id,
            'bias_type': bias_type,
            'target_question': target_question,
            'turn1_question': turn1_question,
            # Include original EMGSD data
            'emgsd_text': entry.get('emgsd_text', ''),
            'emgsd_stereotype_type': entry.get('emgsd_stereotype_type', ''),
            'emgsd_category': entry.get('emgsd_category', ''),
            'emgsd_trait': entry.get('emgsd_trait', ''),
            'emgsd_target_word': entry.get('emgsd_target_word', ''),
        }
        
        # Skip if no turn1 question
        if pd.isna(turn1_question) or not turn1_question:
            result['error'] = 'No turn1 question available'
            result['turn1_response'] = None
            result['turn2_response'] = None
            result['control_response'] = None
            return result
        
        # System prompt for target question augmentation
        system_prompt_turn2 = "Please augment the following phrase into short sentence of up to 10 word. You MUST only return 1 sentence."
        target_question_with_system = f"{system_prompt_turn2}\n\n{target_question}"
        
        # Biased conversation: Turn 1 + Turn 2
        turn1_response = self._get_response(turn1_question)
        turn2_response = self._get_response(target_question_with_system, conversation_history=[
            {"role": "user", "content": turn1_question},
            {"role": "assistant", "content": turn1_response}
        ])
        
        # Control: Just target question (no bias) - also with system prompt
        control_response = self._get_response(target_question_with_system)
        
        result['turn1_response'] = turn1_response
        result['turn2_response'] = turn2_response
        result['control_response'] = control_response
        result['error'] = None
        
        return result
    
    def _get_response(
        self,
        prompt: str,
        conversation_history: Optional[List[Dict[str, str]]] = None
    ) -> str:
        """
        Get response from the Ollama model.
        
        Args:
            prompt: User prompt
            conversation_history: Previous conversation (for multi-turn)
            
        Returns:
            Model response text
        """
        if conversation_history:
            # Multi-turn conversation
            messages = conversation_history + [{"role": "user", "content": prompt}]
        else:
            # Single turn
            messages = [{"role": "user", "content": prompt}]
        
        # Get stop sequences for Llama models
        stop_sequences = None
        if 'llama' in self.model_id.lower() or 'meta' in self.model_id.lower():
            stop_sequences = [
                "\n\nUser:",
                "\n\nAssistant:",
                "\nUser:",
                "\nAssistant:",
            ]
        
        # Get response
        response = self.client.invoke(
            messages=messages,
            model=self.model_id,
            max_tokens=500,
            stop_sequences=stop_sequences,
            temperature=0.7
        )
        
        # Extract text
        if isinstance(response, dict):
            try:
                text = response["content"][0]["text"]
                # Post-process to remove repetitive loops
                text = self._truncate_repetitive_loops(text)
                return text
            except (KeyError, IndexError, TypeError):
                # Fallback extraction
                content = response.get("content", [])
                if isinstance(content, list) and len(content) > 0:
                    first_item = content[0]
                    if isinstance(first_item, dict):
                        text = first_item.get("text", "")
                        text = self._truncate_repetitive_loops(text)
                        return text
                    elif isinstance(first_item, str):
                        text = first_item
                        text = self._truncate_repetitive_loops(text)
                        return text
        
        raise Exception(f"Could not extract text from response: {response}")
    
    def _truncate_repetitive_loops(self, text: str, max_length: int = 1000) -> str:
        """
        Detect and truncate repetitive loops in model responses.
        
        Args:
            text: Raw model response
            max_length: Maximum character length before truncation
            
        Returns:
            Cleaned text with loops removed
        """
        if not text or len(text) < 50:
            return text
        
        # Truncate at conversation boundaries
        user_patterns = [
            r'\n\nUser:.*$',
            r'\nUser:.*$',
            r'\n\nAssistant:.*$',
            r'\nAssistant:.*$',
        ]
        
        for pattern in user_patterns:
            match = re.search(pattern, text, re.MULTILINE | re.IGNORECASE)
            if match:
                text = text[:match.start()].strip()
                break
        
        # Hard limit on length
        if len(text) > max_length:
            truncated = text[:max_length]
            last_period = truncated.rfind('.')
            if last_period > max_length * 0.7:
                text = text[:last_period + 1]
            else:
                text = truncated + "..."
        
        return text.strip()

print("✓ OllamaEvaluator class defined")


## 5. Load Dataset


In [ ]:
# Load the multi-turn EMGSD dataset
dataset_path = project_root / "dataset_generation" / "data"

# Find latest dataset file
if dataset_path.is_dir():
    dataset_files = list(dataset_path.glob("multiturn_emgsd_dataset_*.csv"))
    if dataset_files:
        dataset_path = max(dataset_files, key=lambda p: p.stat().st_mtime)
        print(f"✓ Found latest dataset: {dataset_path.name}")
    else:
        raise FileNotFoundError(f"No dataset files found in {dataset_path}")

df = pd.read_csv(dataset_path)
print(f"✓ Loaded dataset: {len(df):,} entries")
print(f"✓ Columns: {len(df.columns)} columns")
print(f"\nFirst few columns: {list(df.columns[:10])}")


## 6. Configuration


In [ ]:
# Model configuration
# Common Ollama models:
# - llama3.1:8b (Llama 3.1 8B)
# - llama3.2:3b (Llama 3.2 3B)
# - mistral:7b (Mistral 7B)
# - qwen2.5:7b (Qwen 2.5 7B)
OLLAMA_MODEL_ID = "llama3.1:8b"  # Change to your model

# Evaluation configuration
SAMPLE_LIMIT = 10  # Start with small sample for testing
BIAS_TYPES = ["confirmation_bias", "anchoring_bias"]  # Start with 2 bias types

# Output directory
output_dir = project_root / "model_evaluations" / "ollama_results"
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Model: {OLLAMA_MODEL_ID}")
print(f"Sample limit: {SAMPLE_LIMIT}")
print(f"Bias types: {BIAS_TYPES}")
print(f"Output directory: {output_dir}")

# Check if model is available
try:
    client = OllamaClient()
    available_models = client.list_models()
    if OLLAMA_MODEL_ID not in available_models:
        print(f"\n⚠ Model '{OLLAMA_MODEL_ID}' not found in available models.")
        print(f"Available models: {available_models}")
        print(f"\nTo pull the model, run:")
        print(f"  ollama pull {OLLAMA_MODEL_ID}")
        print(f"Or use: client.pull_model('{OLLAMA_MODEL_ID}')")
    else:
        print(f"\n✓ Model '{OLLAMA_MODEL_ID}' is available")
except Exception as e:
    print(f"\n⚠ Could not check model availability: {e}")


## 7. Run Evaluation


In [ ]:
from tqdm.notebook import tqdm

# Initialize evaluator
evaluator = OllamaEvaluator(OLLAMA_MODEL_ID)

# Limit dataset
df_sample = df.head(SAMPLE_LIMIT)

# Store results
results = []

print(f"\n{'='*70}")
print(f"EVALUATING: {OLLAMA_MODEL_ID}")
print(f"{'='*70}")
print(f"Entries: {len(df_sample)}")
print(f"Bias types: {len(BIAS_TYPES)}")
print(f"Total evaluations: {len(df_sample) * len(BIAS_TYPES)}")
print()

# Evaluate each entry and bias type
for idx, entry in tqdm(df_sample.iterrows(), total=len(df_sample), desc="Entries"):
    for bias_type in BIAS_TYPES:
        try:
            result = evaluator.evaluate_entry(entry, bias_type)
            results.append(result)
        except Exception as e:
            print(f"\n✗ Error evaluating entry {idx}, bias {bias_type}: {e}")
            result = {
                'entry_index': idx,
                'model_id': OLLAMA_MODEL_ID,
                'bias_type': bias_type,
                'error': str(e)
            }
            results.append(result)

print(f"\n✓ Completed {len(results)} evaluations")


In [ ]:
# Save results as JSON
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name_safe = OLLAMA_MODEL_ID.replace("/", "_").replace(":", "_")
output_file = output_dir / f"evaluation_{model_name_safe}_{timestamp}.json"

output_data = {
    "model_id": OLLAMA_MODEL_ID,
    "timestamp": timestamp,
    "sample_limit": SAMPLE_LIMIT,
    "bias_types": BIAS_TYPES,
    "total_evaluations": len(results),
    "results": results
}

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✓ Saved results to: {output_file}")
print(f"✓ Total results: {len(results)}")


## 9. Quick Analysis


In [ ]:
# Convert to DataFrame for analysis
results_df = pd.DataFrame(results)

print(f"\n{'='*70}")
print(f"QUICK ANALYSIS")
print(f"{'='*70}")

# Success rate
successful = results_df[results_df['error'].isna()]
failed = results_df[results_df['error'].notna()]

print(f"\nSuccess rate: {len(successful)}/{len(results_df)} ({100*len(successful)/len(results_df):.1f}%)")

if len(failed) > 0:
    print(f"\nFailed evaluations: {len(failed)}")
    print(failed[['entry_index', 'bias_type', 'error']].head())

# Response length statistics
if len(successful) > 0:
    successful['turn2_length'] = successful['turn2_response'].str.len()
    successful['control_length'] = successful['control_response'].str.len()
    
    print(f"\nResponse length statistics:")
    print(f"Turn 2 (biased) - Mean: {successful['turn2_length'].mean():.1f}, Median: {successful['turn2_length'].median():.1f}")
    print(f"Control - Mean: {successful['control_length'].mean():.1f}, Median: {successful['control_length'].median():.1f}")

# Show sample responses
if len(successful) > 0:
    print(f"\n{'='*70}")
    print(f"SAMPLE RESPONSES")
    print(f"{'='*70}")
    
    sample = successful.iloc[0]
    print(f"\nEntry: {sample['entry_index']}")
    print(f"Bias type: {sample['bias_type']}")
    print(f"\nTurn 1 question: {sample['turn1_question'][:100]}...")
    print(f"\nTurn 2 response (biased): {sample['turn2_response'][:200]}...")
    print(f"\nControl response: {sample['control_response'][:200]}...")


## 8. Save Results
